In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gdrive = "/content/drive/"

In [3]:
import os
import shutil

pretrained_model_name = "Mistral-7B-Instruct"
output_dir = gdrive + "MyDrive/" + "outputs_" + pretrained_model_name+ "_finetuned_on_1000_array_basics"
finetuned_model_folder = gdrive + "MyDrive/" + pretrained_model_name + "_finetuned_on_1000_array_basics"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

if os.path.exists(finetuned_model_folder):
    shutil.rmtree(finetuned_model_folder)

In [4]:
%pip install -q -U bitsandbytes
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install -q -U git+https://github.com/huggingface/peft.git
%pip install -q -U git+https://github.com/huggingface/accelerate.git
%pip install -q datasets
%pip install -q scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [6]:
pretrained_model = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(pretrained_model, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, add_eos_token=True)

'''
The following warnings can be ignored. I think they are caused by the fact that the model
is loaded in a single GPU and the parameters are offloaded to the CPU:

WARNING:root:Some parameters are on the meta device device because they were offloaded to the .
WARNING:root:Some parameters are on the meta device device because they were offloaded to the cpu/disk.
'''

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'\nThe following warnings can be ignored. I think they are caused by the fact that the model\nis loaded in a single GPU and the parameters are offloaded to the CPU:\n\nWARNING:root:Some parameters are on the meta device device because they were offloaded to the .\nWARNING:root:Some parameters are on the meta device device because they were offloaded to the cpu/disk.\n'

In [7]:
import pandas as pd
import datasets
datasets.disable_progress_bar()
from datasets import Dataset

# Load the data using pandas
data_file = gdrive + "MyDrive/" + "ARCSolver_core_knowledge_on_basic_arrays_1000.json"
df = pd.read_json(data_file)

# Convert the pandas dataframe to a dataset
data = Dataset.from_pandas(df)

In [8]:
def generate_prompt(data_point):
    text = '<s>[INST] ' + data_point["instruction"] + ' [/INST] ' + str(data_point["output"]) + '</s>'
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in data]
data = data.add_column("prompt", text_column)

In [9]:
data = data.train_test_split(test_size=0.2)
train_data = data["train"]
test_data = data["test"]

In [10]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [11]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)

In [12]:
model.add_adapter(lora_config, adapter_name="adapter")

In [13]:
# Reload the model to avoid the following error. Don't know why, but this has to be done:"
# Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! when resuming training"
model = AutoModelForCausalLM.from_pretrained(pretrained_model, quantization_config=bnb_config, device_map={"":0})

'''
The following warnings can be ignored. I think they are caused by the fact that the model
is loaded in a single GPU and the parameters are offloaded to the CPU:

WARNING:root:Some parameters are on the meta device device because they were offloaded to the .
WARNING:root:Some parameters are on the meta device device because they were offloaded to the cpu/disk.
'''

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'\nThe following warnings can be ignored. I think they are caused by the fact that the model\nis loaded in a single GPU and the parameters are offloaded to the CPU:\n\nWARNING:root:Some parameters are on the meta device device because they were offloaded to the .\nWARNING:root:Some parameters are on the meta device device because they were offloaded to the cpu/disk.\n'

In [14]:
%pip install -q trl
from trl import SFTTrainer

epochs = 4 # 4 was recommended by someone on the OpenAI forum, unless dataset is very small.
per_device_train_batch_size = 2
gradient_accumulation_steps = 4
max_seq_length = 512

steps_per_epoch = len(train_data)//(per_device_train_batch_size*gradient_accumulation_steps)
print("Steps per epoch:", steps_per_epoch)

total_steps = steps_per_epoch * epochs
print("Total steps:", total_steps)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=0.03,
        learning_rate=2e-4,
        logging_steps=steps_per_epoch,
        output_dir=output_dir,
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        evaluation_strategy="epoch",
        num_train_epochs=epochs,
        save_steps=steps_per_epoch,
        load_best_model_at_end=True,
        fp16=True,
        gradient_checkpointing=True,
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

'''
The following warning can be ignored as far as I can tell. Via web search: it happens to many people,
and no fix has been found yet:

UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer.
This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision.
You might consider adding `tokenizer.padding_side = 'right'` to your code.
'''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 14.9 MB/s eta 0:00:00
Steps per epoch: 100
Total steps: 400


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


"\nThe following warning can be ignored as far as I can tell. Via web search: it happens to many people,\nand no fix has been found yet:\n\nUserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer.\nThis might lead to some unexpected behaviour due to overflow issues when training a model in half-precision.\nYou might consider adding `tokenizer.padding_side = 'right'` to your code.\n"

In [15]:
model.config.use_cache = False  # silence the warnings.
trainer.train()

'''
The following warning can be ignored as far as I can tell. Via web search: it happens to many people
and no fix has been found yet:

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer,
using the `__call__` method is faster than using a method to encode the text followed
by a call to the `pad` method to get a padded encoding.
'''

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.631300,0.497024
2,0.472600,0.495935
3,0.466000,0.496126
4,0.466400,0.495495


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

"\nThe following warning can be ignored as far as I can tell. Via web search: it happens to many people\nand no fix has been found yet:\n\nYou're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer,\nusing the `__call__` method is faster than using a method to encode the text followed\nby a call to the `pad` method to get a padded encoding.\n"

In [16]:
trainer.save_model(finetuned_model_folder)